# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

training_file = "data/train.p"
validation_file = "data/valid.p"
testing_file = "data/test.p"

# Load training-, validation-, and test-data
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

# split data into images and labels
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
# Print some information on the data

n_train = len(X_train)
n_test = len(X_test)
n_classes = len(set(y_train))
image_shape = X_train[0].shape

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

In [ ]:
import matplotlib.pyplot as plt
from numpy import where
import pandas as pd

# Visualizations will be shown in the notebook.
%matplotlib inline

# Load class names
classes = pd.read_csv('signnames.csv', sep=',',header=1)

def previewImage(image, label, name):
    print(label, " - ", name)
    plt.figure(figsize = (1,1))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# for each class find and render 1 example from test set
for label, name in classes.values:
    idx = where(y_test == label)[0][0]
    previewImage(X_test[idx], label, name)


----

## Step 2: Design a Model Architecture

### Pre-process the Data Set (normalization and rotated duplication)

In [ ]:
from numpy import subtract
from numpy import divide
import cv2

def normalize(images):
    images_prep = []
    for image in images:
        images_prep.append(divide(subtract(image, 128.0), 128.0))
    return images_prep

def rotate(image, angle):
    M = cv2.getRotationMatrix2D((image.shape[0] / 2, image.shape[1] / 2), angle, 1)
    return cv2.warpAffine(image, M, (image.shape[0], image.shape[1]))

# increase number of samples for training by modifying them slightly
def duplicate(images, labels):
    images_dup = []
    labels_dup = []
    for image in images:
        # original
        images_dup.append(image)
        # rotate +5
        images_dup.append(rotate(image, 5))
        # rotate -5
        images_dup.append(rotate(image, -5))
        # rotate +10
        images_dup.append(rotate(image, 10))
        # rotate -10
        images_dup.append(rotate(image, -10))
    for label in labels:
        for duplicates in range(5):
            labels_dup.append(label)
    return images_dup, labels_dup

X_train, y_train = duplicate(X_train, y_train)
print("New number of training samples:", len(X_train))
X_train = normalize(X_train)
X_valid = normalize(X_valid)
X_test = normalize(X_test)

### Model Architecture

In [ ]:
import tensorflow as tf

EPOCHS = 10
BATCH_SIZE = 256

In [ ]:
# implementation of the actual network
from tensorflow.contrib.layers import flatten

def SignNet(sign, shape, num_labels):
    # constants used for tf.truncated_normal
    mu = 0
    sigma = 0.1    
    
    # Layer 1: Convolutional.
    # Input = 32x32x3 (width x height  x 3)
    # Output = 28x28x6 (width-4) x (height-4) x 6
    # filter with size=5x5, stride=1
    l1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean=mu, stddev=sigma))
    l1_b = tf.Variable(tf.zeros(6))
    l1_strides = [1, 1, 1, 1]
    l1 = tf.nn.conv2d(sign, l1_W, strides=l1_strides, padding='VALID') + l1_b
    l1 = tf.nn.relu(l1)

    # maximum pooling to reduce size to 14x14x6 / (width-4)/2 x (height-4)/2 x 6
    l1p_kernel = [1, 2, 2, 1]
    l1p_strides = [1, 2, 2, 1]
    l1p = tf.nn.max_pool(l1, ksize=l1p_kernel, strides=l1p_strides, padding='VALID')    
    
    # Layer 2: Convolutional.
    # Input = (width-4)/2 x (height-4)/2 x 6
    # Output = ((width-4)/2-4) x ((height-4)/2-4) x 16
    l2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean=mu, stddev=sigma))
    l2_b = tf.Variable(tf.zeros(16))
    l2_strides = [1, 1, 1, 1]
    l2 = tf.nn.conv2d(l1p, l2_W, strides=l2_strides, padding='VALID') + l2_b
    l2 = tf.nn.relu(l2)
    
    # maximum pooling to reduce size to 5x5x16 / ((width-4)/2-4)/2 x ((height-4)/2-4)/2 x 16
    l2p_kernel = [1, 2, 2, 1]
    l2p_strides = [1, 2, 2, 1]
    l2p = tf.nn.max_pool(l2, ksize=l2p_kernel, strides=l2p_strides, padding='VALID')
    
    # Flatten.
    # Input = 5x5x16 / ((width-4)/2-4)/2 x ((height-4)/2-4)/2 x 16
    # Output = 400 / ((width-4)/2-4)/2 * ((height-4)/2-4)/2 * 16
    flat = flatten(l2p)
    flat_len = 400 # FIXME

    # Layer 3: Fully Connected, cut to 2/3
    # Input = 400 / ((width-4)/2-4)/2 * ((height-4)/2-4)/2 * 16
    # Output = 200 / ((width-4)/2-4)/2 * ((height-4)/2-4)/2 * 16 / 2
    l3_len = 267 # FIXME
    l3_W = tf.Variable(tf.truncated_normal(shape=(flat_len, l3_len), mean=mu, stddev=sigma))
    l3_b = tf.Variable(tf.zeros(l3_len))
    l3 = tf.add(tf.matmul(flat, l3_W), l3_b)
    l3 = tf.nn.relu(l3)

    # Layer 4: Fully Connected, vut to 2/3
    # Input = 200 / ((width-4)/2-4)/2 * ((height-4)/2-4)/2 * 16 / 2
    # Input = 100 / ((width-4)/2-4)/2 * ((height-4)/2-4)/2 * 16 / 2 / 2
    l4_len = 178 # FIXME
    l4_W = tf.Variable(tf.truncated_normal(shape=(l3_len, l4_len), mean=mu, stddev=sigma))
    l4_b = tf.Variable(tf.zeros(l4_len))
    l4 = tf.add(tf.matmul(l3, l4_W), l4_b)
    l4 = tf.nn.relu(l4)

    # Layer 5: Fully Connected. Input = 84. Output = 10.
    l5_W = tf.Variable(tf.truncated_normal(shape=(l4_len, num_labels), mean=mu, stddev=sigma))
    l5_b = tf.Variable(tf.zeros(num_labels))
    logits = tf.add(tf.matmul(l4, l5_W), l5_b)

    return logits

In [ ]:
# placeholders for batches(x=input images, y=output labels)

sign = tf.placeholder(tf.float32, (None, image_shape[0], image_shape[1], image_shape[2]))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

In [ ]:
# training pipeline

rate = 0.001
logits = SignNet(sign, image_shape, n_classes)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={sign: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return 100 * total_accuracy / num_examples

### Train and Validate the Model

In [ ]:
from sklearn.utils import shuffle

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={sign: batch_x, y: batch_y})

        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:2.2f}%".format(validation_accuracy))
        print()
        
    saver.save(sess, './network')
    print("Model saved")

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 